In [ ]:
# Refractive index reconstruction script Vargas et al.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

def calcular_indice_complejo(h: float, r_file_path: str, t_file_path: str):
    """
    Calcula el índice de refracción complejo (n y k) a partir de archivos de reflectancia y transmitancia,
    genera el plot y lo guarda como PNG.

    Parámetros
    ----------
    h : float
        Espesor de la muestra (en las mismas unidades que la longitud de onda de los datos, ej. nm o µm).
    r_file_path : str
        Ruta al archivo de reflectancia (.ASC).
    t_file_path : str
        Ruta al archivo de transmitancia (.ASC).

    Retorna
    -------
    df_result : pd.DataFrame
        DataFrame con columnas ['wavelength', 'R', 'T', 'r', 'k', 'n']
    fig : matplotlib.figure.Figure
        Figura con las gráficas de n y k vs longitud de onda.
    """

    # === Cargar archivos ===
    t_df = pd.read_csv(
        t_file_path,
        header=90,
        names=["wavelength", "T"],
        decimal=",",
        sep="\t",
        dtype=float
    )

    r_df = pd.read_csv(
        r_file_path,
        header=90,
        names=["wavelength", "R"],
        decimal=",",
        sep="\t",
        dtype=float
    )

    # Normalizar R y T
    r_df["R"] = r_df["R"] / 100
    t_df["T"] = t_df["T"] / 100

    # Unir por longitud de onda
    df = pd.merge(r_df, t_df, on="wavelength", how="inner")

    # === Cálculo de n y k ===
    R = df["R"].to_numpy()
    T = df["T"].to_numpy()
    wavelength = df["wavelength"].to_numpy()

    # (11a) r
    term1 = (T**2 + 2 - (R - 1)**2) / (2 * (2 - R))
    r = term1 - np.sqrt(term1**2 - R / (2 - R))

    # (11b) k
    ratio = (R - r) / (r * T)
    ratio = np.clip(ratio, 1e-20, None)
    k = -(wavelength / (4 * np.pi * h)) * np.log(ratio)

    # (11c) n
    frac = (1 - r) / (1 + r)
    inside_sqrt = 1 - (frac**2) * (1 + k**2)
    inside_sqrt = np.clip(inside_sqrt, 0, None)
    n = (1 + r) / (1 - r) * (1 + np.sqrt(inside_sqrt))

    # Guardar en DataFrame
    df["r"] = r
    df["k"] = k
    df["n"] = n

    # === Graficar ===
    fig, ax = plt.subplots(figsize=(8, 5))
    ax.plot(df["wavelength"], df["n"], marker="o", label="n (refractive index)")
    ax.plot(df["wavelength"], df["k"], marker="s", label="k (extinction coefficient)")
    ax.set_xlabel("Wavelength (nm)")
    ax.set_ylabel("Value")
    ax.set_title("n and k vs Wavelength")
    ax.legend()
    ax.grid(True)

    # === Guardar gráfico ===
    base_name = os.path.splitext(os.path.basename(r_file_path))[0]
    output_dir = os.path.dirname(r_file_path)
    output_path = os.path.join(output_dir, f"{base_name}_n_k_plot.png")
    fig.savefig(output_path, dpi=300, bbox_inches="tight")

    print(f"✅ Gráfico guardado en: {output_path}")

    return df, fig


In [1]:
from pathlib import Path

base_folder = Path(r"C:\Users\EstebanSoto\Jupyter\escarabajos\L1050_data\TRA_data_CICIMA_INBUCR\CICIMAUCR")
reflectance = base_folder / "reflectance"
transmittance = base_folder / "transmittance"
thickness_file = base_folder / "thickness" / "thickness.txt"



In [2]:
import os

def emparejar_archivos(reflectance_dir: str, transmittance_dir: str):
    """
    Busca archivos en las carpetas de reflectancia y transmitancia,
    y devuelve un diccionario con los códigos y sus rutas.

    Parámetros
    ----------
    reflectance_dir : str
        Ruta a la carpeta que contiene archivos de reflectancia.
    transmittance_dir : str
        Ruta a la carpeta que contiene archivos de transmitancia.

    Retorna
    -------
    dict
        Diccionario con estructura:
        {
            codigo: {
                "path_r": "ruta/al/archivo_r.txt",
                "path_t": "ruta/al/archivo_t.txt"
            },
            ...
        }
    """
    # Listar archivos de cada carpeta
    files_r = {os.path.splitext(f)[0]: os.path.join(reflectance_dir, f)
               for f in os.listdir(reflectance_dir) if f.lower().endswith(".txt")}
    
    files_t = {os.path.splitext(f)[0]: os.path.join(transmittance_dir, f)
               for f in os.listdir(transmittance_dir) if f.lower().endswith(".txt")}
    
    # Emparejar por código
    codigos_comunes = set(files_r.keys()) & set(files_t.keys())
    
    resultado = {}
    for codigo in sorted(codigos_comunes):
        resultado[codigo] = {
            "path_r": files_r[codigo],
            "path_t": files_t[codigo]
        }
    
    return resultado


In [3]:
import pandas as pd

import pandas as pd
import warnings

def agregar_thickness(diccionario: dict, thickness_file: str):
    """
    Añade la información de espesor ('thickness') al diccionario de códigos.

    Parámetros
    ----------
    diccionario : dict
        Diccionario con estructura:
        {
            codigo: {
                "path_r": "...",
                "path_t": "..."
            },
            ...
        }
    thickness_file : str
        Ruta al archivo .txt con columnas ['code', 'thickness'].

    Retorna
    -------
    dict
        El mismo diccionario ampliado con un campo 'thickness' para cada código.
    """

    # Leer archivo de espesores
    df = pd.read_csv(thickness_file, sep="\t", dtype={"code": str})

    # Crear un diccionario {codigo: thickness}
    thickness_dict = dict(zip(df["code"], df["thickness"]))

    # Agregar al diccionario original
    for codigo in diccionario.keys():
        if codigo in thickness_dict:
            diccionario[codigo]["thickness"] = thickness_dict[codigo]
        else:
            diccionario[codigo]["thickness"] = None
            warnings.warn(f"⚠️ No se encontró thickness para el código: {codigo}")

    return diccionario


In [ ]:
def procesar_indices(diccionario: dict):
    """
    Para cada código en el diccionario, calcula el índice complejo (n y k)
    usando calcular_indice_complejo y añade los resultados al diccionario.

    Parámetros
    ----------
    diccionario : dict
        Diccionario con estructura:
        {
            codigo: {
                "path_r": "...",
                "path_t": "...",
                "thickness": valor
            },
            ...
        }

    Retorna
    -------
    dict
        Diccionario ampliado con llaves extra:
        {
            codigo: {
                "path_r": "...",
                "path_t": "...",
                "thickness": valor,
                "df": DataFrame con resultados,
                "fig": objeto Figure con gráfico
            }
        }
    """
    for codigo, info in diccionario.items():
        h = info.get("thickness", None)

        if h is None:
            print(f"⚠️ Saltando {codigo}: no tiene thickness definido.")
            continue

        try:
            df, fig = calcular_indice_complejo(
                h=h,
                r_file_path=info["path_r"],
                t_file_path=info["path_t"]
            )
            diccionario[codigo]["df"] = df
            diccionario[codigo]["fig"] = fig
        except Exception as e:
            print(f"❌ Error procesando {codigo}: {e}")
            diccionario[codigo]["df"] = None
            diccionario[codigo]["fig"] = None

    return diccionario


In [ ]:
import os

def guardar_resultados(diccionario: dict, base_dir: str = "indices_de_refraccion_complejos", extension: str = "csv"):
    """
    Guarda los DataFrames y plots de cada código en carpetas organizadas.

    Parámetros
    ----------
    diccionario : dict
        Diccionario con estructura:
        {
            codigo: {
                "df": DataFrame con resultados,
                "fig": matplotlib.figure.Figure,
                ...
            },
            ...
        }
    base_dir : str
        Carpeta raíz donde se crearán las subcarpetas.
    extension : str
        Extensión para guardar los DataFrames ("csv" o "xlsx").
    """

    # Crear estructura de carpetas
    dir_indices = os.path.join(base_dir, "indice_refraccion")
    dir_plots = os.path.join(base_dir, "plots")
    os.makedirs(dir_indices, exist_ok=True)
    os.makedirs(dir_plots, exist_ok=True)

    for codigo, info in diccionario.items():
        df = info.get("df")
        fig = info.get("fig")

        if df is None or fig is None:
            print(f"⚠️ Saltando {codigo}: no tiene resultados para guardar.")
            continue

        # Guardar DataFrame
        if extension == "csv":
            df.to_csv(os.path.join(dir_indices, f"{codigo}.csv"), index=False)
        elif extension == "xlsx":
            df.to_excel(os.path.join(dir_indices, f"{codigo}.xlsx"), index=False)
        else:
            raise ValueError("Extensión no soportada. Usa 'csv' o 'xlsx'.")

        # Guardar gráfico
        fig.savefig(os.path.join(dir_plots, f"{codigo}.png"), dpi=300, bbox_inches="tight")

        print(f"✅ Guardados resultados para {codigo}")

    print(f"\n📂 Todo guardado en: {os.path.abspath(base_dir)}")
